In [1]:
import math
import pandas as pd
import numpy as np
import warnings
from sklearn.metrics import confusion_matrix
import math
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

import Feature_rank_lib as lib

In [2]:
# result_exp3 = pd.read_csv('./results/Exp3/RF_Exp3_24features_wsALL_cwALL_bootstrap10.csv')
result_exp3_100 = pd.read_csv('./results/Exp3/RF_Exp3_24features_wsALL_cwALL_bootstrap100.csv')
result_exp3_1000 = pd.read_csv('./results/Exp3/RF_Exp3_24features_wsALL_cwALL_bootstrap1000.csv')

exp3_columns = np.load('./info/Exp3_all_columns.npy',allow_pickle=True)

In [3]:
log2_exp3 = math.ceil(np.log2(exp3_columns.shape[0]))
sqrt_exp3 = math.ceil(np.sqrt(exp3_columns.shape[0]))

In [4]:
log2_top_exp3_100 = lib.feature_top(result_exp3_100,exp3_columns,log2_exp3)
log2_top_exp3_1000 = lib.feature_top(result_exp3_1000,exp3_columns,log2_exp3)

sqrt_top_exp3_100 = lib.feature_top(result_exp3_100,exp3_columns,sqrt_exp3)
sqrt_top_exp3_1000 = lib.feature_top(result_exp3_1000,exp3_columns,sqrt_exp3)

In [5]:
log2_top_exp3_100.to_csv('./data/Important_Features/log2_top_exp3_bs100.csv')
log2_top_exp3_1000.to_csv('./data/Important_Features/log2_top_exp3_bs1000.csv')
sqrt_top_exp3_100.to_csv('./data/Important_Features/sqrt_top_exp3_100.csv')
sqrt_top_exp3_1000.to_csv('./data/Important_Features/sqrt_top_exp3_1000.csv')

In [24]:
import xgboost as xgb
from xgboost import XGBClassifier
import Measurements as measurements
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)

In [6]:
exp3_p1 = pd.read_csv('./data/exp3/exp3_p1_wsALL_24features.csv')
exp3_p2 = pd.read_csv('./data/exp3/exp3_p2_wsALL_24features.csv')
exp3_p3 = pd.read_csv('./data/exp3/exp3_p3_wsALL_24features.csv')
exp3_p4 = pd.read_csv('./data/exp3/exp3_p4_wsALL_24features.csv')
exp3_p5 = pd.read_csv('./data/exp3/exp3_p5_wsALL_24features.csv')

In [190]:
import time
import pickle
import warnings
import Measurements as measurements
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [191]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'scale_pos_weight': hp.choice('scale_pos_weight', [1,3,4,5,6,7,8,9,10,20,30]),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': hp.quniform('n_estimators', 10, 100, 1)
    }

hp_df = pd.DataFrame(columns=['colsample_bytree', 'gamma', 'max_depth', 'min_child_weight', 'n_estimators', 'reg_alpha', 'reg_lambda', 'scale_pos_weight','TSS','HSS','Confusion_Matrix(tn, fp, fn, tp)'])

In [192]:
def objective_weighted(space):
    clf=xgb.XGBClassifier(
                    n_estimators = int(space['n_estimators']), max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight = int(space['min_child_weight']), scale_pos_weight = space['scale_pos_weight'],
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="logloss",
            early_stopping_rounds=10,verbose=False)
    

    y_pred = clf.predict(X_test)
    scores = confusion_matrix(y_test, y_pred, labels=[0, 1]).ravel()
    tss = measurements.TSS(scores)
    hss = measurements.HSS2(scores)
    weighted = (tss+hss)/2
#     weighted_sqrt = math.sqrt(tss*hss)
    output_file.write("SCORE: " + str(weighted) + '\n')
    output_file.write("TSS: " + str(tss) + "  HSS: " + str(hss) + '\n')
    return {'TSS': tss, 'HSS': hss, 'Confusion_Matrix(tn, fp, fn, tp)': [scores], 'score': weighted, 'loss': -weighted, 'status': STATUS_OK}

def objective_weighted_TSS(space):
    clf=xgb.XGBClassifier(
                    n_estimators = int(space['n_estimators']), max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']), scale_pos_weight=space['scale_pos_weight'],
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="logloss",
            early_stopping_rounds=10,verbose=False)
    

    y_pred = clf.predict(X_test)
    scores = confusion_matrix(y_test, y_pred, labels=[0, 1]).ravel()
    tss = measurements.TSS(scores)
    hss = measurements.HSS2(scores)
    tpr = measurements.TPR(scores)
    tnr = measurements.TNR(scores)
    weighted = 0.5*tpr+1.5*tnr-1
    output_file.write("SCORE: " + str(weighted) + '\n')
    output_file.write("TSS: " + str(tss) + "  HSS: " + str(hss) + '\n')
    return {'TSS': tss, 'HSS': hss, 'Confusion_Matrix(tn, fp, fn, tp)': [scores], 'score': weighted, 'loss': -weighted, 'status': STATUS_OK }

def objective_weighted_TSS2(space):
    clf=xgb.XGBClassifier(
                    n_estimators = int(space['n_estimators']), max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']), scale_pos_weight=space['scale_pos_weight'],
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="logloss",
            early_stopping_rounds=10,verbose=False)
    

    y_pred = clf.predict(X_test)
    scores = confusion_matrix(y_test, y_pred, labels=[0, 1]).ravel()
    tss = measurements.TSS(scores)
    hss = measurements.HSS2(scores)
    tpr = measurements.TPR(scores)
    tnr = measurements.TNR(scores)
    weighted = 0.75*tpr+1.25*tnr-1
    output_file.write("SCORE: " + str(weighted) + '\n')
    output_file.write("TSS: " + str(tss) + "  HSS: " + str(hss) + '\n')
    return {'TSS': tss, 'HSS': hss, 'Confusion_Matrix(tn, fp, fn, tp)': [scores], 'score': weighted, 'loss': -weighted, 'status': STATUS_OK }

def objective_weighted_TSS3(space):
    clf=xgb.XGBClassifier(
                    n_estimators = int(space['n_estimators']), max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']), scale_pos_weight=space['scale_pos_weight'],
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="logloss",
            early_stopping_rounds=10,verbose=False)
    

    y_pred = clf.predict(X_test)
    scores = confusion_matrix(y_test, y_pred, labels=[0, 1]).ravel()
    tss = measurements.TSS(scores)
    hss = measurements.HSS2(scores)
    tpr = measurements.TPR(scores)
    tnr = measurements.TNR(scores)
    weighted = 0.2*tpr+1.8*tnr-1
    output_file.write("SCORE: " + str(weighted) + '\n')
    output_file.write("TSS: " + str(tss) + "  HSS: " + str(hss) + '\n')
    return {'TSS': tss, 'HSS': hss, 'Confusion_Matrix(tn, fp, fn, tp)': [scores], 'score': weighted, 'loss': -weighted, 'status': STATUS_OK }

def objective_weighted_TSS4(space):
    clf=xgb.XGBClassifier(
                    n_estimators = int(space['n_estimators']), max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']), scale_pos_weight=space['scale_pos_weight'],
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="logloss",
            early_stopping_rounds=10,verbose=False)
    

    y_pred = clf.predict(X_test)
    scores = confusion_matrix(y_test, y_pred, labels=[0, 1]).ravel()
    tss = measurements.TSS(scores)
    hss = measurements.HSS2(scores)
    tpr = measurements.TPR(scores)
    tnr = measurements.TNR(scores)
    weighted = 1.5*tpr+0.5*tnr-1
    output_file.write("SCORE: " + str(weighted) + '\n')
    output_file.write("TSS: " + str(tss) + "  HSS: " + str(hss) + '\n')
    return {'TSS': tss, 'HSS': hss, 'Confusion_Matrix(tn, fp, fn, tp)': [scores], 'score': weighted, 'loss': -weighted, 'status': STATUS_OK }


def getBestModelfromTrials(trials):
    valid_trial_list = [trial for trial in trials
                            if STATUS_OK == trial['result']['status']]
    losses = [ float(trial['result']['loss']) for trial in valid_trial_list]
    index_having_minumum_loss = np.argmin(losses)
    best_trial_obj = valid_trial_list[index_having_minumum_loss]
    return best_trial_obj['result']['TSS'], best_trial_obj['result']['HSS'], best_trial_obj['result']['Confusion_Matrix(tn, fp, fn, tp)'], best_trial_obj['result']['score']

# Validation set generate
## Testing on p5

In [193]:
# Train on p123 (or p124/p134/p234)
# Validate on p4 (or p3/p2/p1)
# Test on p5

train_set_exp3_TEp5_VALp4 = [exp3_p1,exp3_p2,exp3_p3]
train_set_exp3_TEp5_VALp3 = [exp3_p1,exp3_p2,exp3_p4]
train_set_exp3_TEp5_VALp2 = [exp3_p1,exp3_p3,exp3_p4]
train_set_exp3_TEp5_VALp1 = [exp3_p2,exp3_p3,exp3_p4]

valid_set_exp3_TEp5_VALp4 = [exp3_p4]
valid_set_exp3_TEp5_VALp3 = [exp3_p3]
valid_set_exp3_TEp5_VALp2 = [exp3_p2]
valid_set_exp3_TEp5_VALp1 = [exp3_p1]

log_exp3_X_train_100_list = []
sqrt_exp3_X_train_100_list = []
log_exp3_X_test_100_list = []
sqrt_exp3_X_test_100_list = []

exp3_y_train_100_list = []
exp3_y_test_100_list = []

# generate training set for using validation set p4
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp5_VALp4,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp5_VALp4,sqrt_top_exp3_100['Features'].to_list())))
# p3
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp5_VALp3,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp5_VALp3,sqrt_top_exp3_100['Features'].to_list())))
# p2
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp5_VALp2,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp5_VALp2,sqrt_top_exp3_100['Features'].to_list())))
# p1
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp5_VALp1,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp5_VALp1,sqrt_top_exp3_100['Features'].to_list())))

# validation set p4
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp5_VALp4,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp5_VALp4,sqrt_top_exp3_100['Features'].to_list())))
# validation set p3
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp5_VALp3,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp5_VALp3,sqrt_top_exp3_100['Features'].to_list())))
# validation set p2
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp5_VALp2,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp5_VALp2,sqrt_top_exp3_100['Features'].to_list())))
# validation set p1
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp5_VALp1,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp5_VALp1,sqrt_top_exp3_100['Features'].to_list())))


exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp5_VALp4))
exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp5_VALp3))
exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp5_VALp2))
exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp5_VALp1))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp5_VALp4))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp5_VALp3))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp5_VALp2))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp5_VALp1))

In [194]:
train_exp3_TRp1234 = [exp3_p1,exp3_p2,exp3_p3,exp3_p4]
test_exp3_TEp5 = [exp3_p5]

log_exp3_X_train_100_TRp1234 = lib.rename_columns(lib.extract_data(train_exp3_TRp1234,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_train_100_TRp1234 = lib.rename_columns(lib.extract_data(train_exp3_TRp1234,sqrt_top_exp3_100['Features'].to_list()))
log_exp3_X_test_100_TEp5 = lib.rename_columns(lib.extract_data(test_exp3_TEp5,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_test_100_TEp5 = lib.rename_columns(lib.extract_data(test_exp3_TEp5,sqrt_top_exp3_100['Features'].to_list()))

exp3_y_train_100_TRp1234 = lib.extract_label(train_exp3_TRp1234)
exp3_y_test_100_TEp5 = lib.extract_label(test_exp3_TEp5)

In [195]:
output_file = open('./XGBobjective_weight_output_TEp5.txt', 'w')
for i in range(4):
    X_train, y_train, X_test, y_test = log_exp3_X_train_100_list[i], exp3_y_train_100_list[i], log_exp3_X_test_100_list[i], exp3_y_test_100_list[i]
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)

    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp'+str(i+1)+'_averageTSSHSS'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log3_exp3_boostrap100_TEp'+str(i+1)+'_0.5TPR_1.5TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS2,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp'+str(i+1)+'_0.75TPR_1.25TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS3,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp'+str(i+1)+'_0.2TPR_1.8TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS4,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp'+str(i+1)+'_1.5TPR_0.5TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)
    
output_file.close()

100%|████████| 50/50 [18:44<00:00, 22.48s/trial, best loss: -0.7549790979772595]


In [196]:
hp_df.to_csv('./hp_df_TEp5.csv')

In [197]:
hp_df

,colsample_bytree,gamma,max_depth,min_child_weight,n_estimators,reg_alpha,reg_lambda,scale_pos_weight,TSS,HSS,"Confusion_Matrix(tn, fp, fn, tp)",score,exp
0,0.641377,7.079832,12.0,7.0,19.0,52.0,0.902954,9,0.809080,0.316691,"[[46212, 3842, 133, 1032]]",0.562885,log2_exp3_boostrap100_TEp1_averageTSSHSS
1,0.927130,4.046318,14.0,5.0,15.0,80.0,0.183584,9,0.828320,0.312728,"[[46015, 4039, 106, 1059]]",0.833467,log3_exp3_boostrap100_TEp1_0.5TPR_1.5TNR
2,0.908068,8.845125,3.0,4.0,12.0,72.0,0.007414,9,0.838896,0.304110,"[[45771, 4283, 88, 1077]]",0.836388,log2_exp3_boostrap100_TEp1_0.75TPR_1.25TNR
3,0.506493,8.786948,6.0,3.0,36.0,106.0,0.009724,3,0.595809,0.459004,"[[48899, 1155, 444, 721]]",0.882242,log2_exp3_boostrap100_TEp1_0.2TPR_1.8TNR
4,0.968303,1.458106,8.0,6.0,37.0,122.0,0.190331,10,0.829224,0.274873,"[[45158, 4896, 85, 1080]]",0.841651,log2_exp3_boostrap100_TEp1_1.5TPR_0.5TNR
5,0.782940,6.577603,9.0,6.0,12.0,179.0,0.040286,10,0.785236,0.290338,"[[35902, 5156, 127, 1297]]",0.537787,log2_exp3_boostrap100_TEp2_averageTSSHSS
6,0.612135,1.021725,8.0,1.0,83.0,179.0,0.175087,7,0.693291,0.358954,"[[37980, 3078, 330, 1094]]",0.771679,log3_exp3_boostrap100_TEp2_0.5TPR_1.5TNR
7,0.934133,4.562100,5.0,1.0,76.0,174.0,0.748357,10,0.785114,0.290111,"[[35897, 5161, 127, 1297]]",0.775986,log2_exp3_boostrap100_TEp2_0.75TPR_1.25TNR
8,0.993868,5.234716,4.0,8.0,41.0,59.0,0.315149,1,0.455186,0.421869,"[[40083, 975, 742, 682]]",0.853042,log2_exp3_boostrap100_TEp2_0.2TPR_1.8TNR
9,0.977867,2.612095,7.0,6.0,15.0,53.0,0.436743,10,0.779886,0.295697,"[[36086, 4972, 141, 1283]]",0.790926,log2_exp3_boostrap100_TEp2_1.5TPR_0.5TNR


In [198]:
bst = XGBClassifier(n_estimators =int(hp_df['n_estimators'][3]), max_depth = int(hp_df['max_depth'][3]), gamma = hp_df['gamma'][3],
                reg_alpha = int(hp_df['reg_alpha'][3]),min_child_weight=int(hp_df['min_child_weight'][3]), scale_pos_weight=hp_df['scale_pos_weight'][3],
                colsample_bytree=int(hp_df['colsample_bytree'][3]))
t0=time.time()
bst.fit(log_exp3_X_train_100_TRp1234, exp3_y_train_100_TRp1234,
        eval_set=evaluation, eval_metric="logloss",
        early_stopping_rounds=10,verbose=False)
print("\tTraining time:", round(time.time()-t0, 2), "s") # the time would be round to 3 decimal in seconds
t1=time.time()
exp_y_pred = bst.predict(log_exp3_X_test_100_TEp5)
print("\tPredicting time:", round(time.time()-t1, 2), "s") # the time would be round to 3 decimal in seconds
scores = confusion_matrix(exp3_y_test_100_TEp5, exp_y_pred, labels=[0, 1]).ravel()
measurements.TSS(scores), measurements.HSS2(scores), [scores]

	Training time: 11.21 s
	Predicting time: 0.02 s


(0.41331122318444324,
 0.4264879589499852,
 [array([73790,   512,   574,   416])])

## Testing on p4

In [200]:
# Train on p123 (or p124/p134/p234)
# Validate on p4 (or p3/p2/p1)
# Test on p5

train_set_exp3_TEp4_VALp5 = [exp3_p1,exp3_p2,exp3_p3]
train_set_exp3_TEp4_VALp3 = [exp3_p1,exp3_p2,exp3_p5]
train_set_exp3_TEp4_VALp2 = [exp3_p1,exp3_p3,exp3_p5]
train_set_exp3_TEp4_VALp1 = [exp3_p2,exp3_p3,exp3_p5]

valid_set_exp3_TEp4_VALp5 = [exp3_p5]
valid_set_exp3_TEp4_VALp3 = [exp3_p3]
valid_set_exp3_TEp4_VALp2 = [exp3_p2]
valid_set_exp3_TEp4_VALp1 = [exp3_p1]

log_exp3_X_train_100_list = []
sqrt_exp3_X_train_100_list = []
log_exp3_X_test_100_list = []
sqrt_exp3_X_test_100_list = []

exp3_y_train_100_list = []
exp3_y_test_100_list = []

# generate training set for using validation set p4
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp4_VALp5,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp4_VALp5,sqrt_top_exp3_100['Features'].to_list())))
# p3
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp4_VALp3,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp4_VALp3,sqrt_top_exp3_100['Features'].to_list())))
# p2
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp4_VALp2,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp4_VALp2,sqrt_top_exp3_100['Features'].to_list())))
# p1
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp4_VALp1,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp4_VALp1,sqrt_top_exp3_100['Features'].to_list())))

# validation set p4
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp4_VALp5,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp4_VALp5,sqrt_top_exp3_100['Features'].to_list())))
# validation set p3
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp4_VALp3,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp4_VALp3,sqrt_top_exp3_100['Features'].to_list())))
# validation set p2
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp4_VALp2,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp4_VALp2,sqrt_top_exp3_100['Features'].to_list())))
# validation set p1
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp4_VALp1,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp4_VALp1,sqrt_top_exp3_100['Features'].to_list())))


exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp4_VALp5))
exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp4_VALp3))
exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp4_VALp2))
exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp4_VALp1))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp4_VALp5))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp4_VALp3))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp4_VALp2))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp4_VALp1))

In [203]:
train_exp3_TRp1235 = [exp3_p1,exp3_p2,exp3_p3,exp3_p5]
test_exp3_TEp4 = [exp3_p4]

log_exp3_X_train_100_TRp1235 = lib.rename_columns(lib.extract_data(train_exp3_TRp1235,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_train_100_TRp1235 = lib.rename_columns(lib.extract_data(train_exp3_TRp1235,sqrt_top_exp3_100['Features'].to_list()))
log_exp3_X_test_100_TEp4 = lib.rename_columns(lib.extract_data(test_exp3_TEp4,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_test_100_TEp4 = lib.rename_columns(lib.extract_data(test_exp3_TEp4,sqrt_top_exp3_100['Features'].to_list()))

exp3_y_train_100_TRp1235 = lib.extract_label(train_exp3_TRp1235)
exp3_y_test_100_TEp4 = lib.extract_label(test_exp3_TEp4)

In [205]:
output_file = open('./XGBobjective_weight_output_TEp4.txt', 'w')
for i in range(4):
    X_train, y_train, X_test, y_test = log_exp3_X_train_100_list[i], exp3_y_train_100_list[i], log_exp3_X_test_100_list[i], exp3_y_test_100_list[i]
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)

    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp4_VALp'+str(i+1)+'_averageTSSHSS'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp4_VALp'+str(i+1)+'_0.5TPR_1.5TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS2,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp4_VALp'+str(i+1)+'_0.75TPR_1.25TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS3,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp4_VALp'+str(i+1)+'_0.2TPR_1.8TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS4,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp4_VALp'+str(i+1)+'_1.5TPR_0.5TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)
    
output_file.close()

100%|████████| 50/50 [12:14<00:00, 14.69s/trial, best loss: -0.7742709331008424]


## Testing on p3

In [206]:
train_set_exp3_TEp3_VALp5 = [exp3_p1,exp3_p2,exp3_p4]
train_set_exp3_TEp3_VALp4 = [exp3_p1,exp3_p2,exp3_p5]
train_set_exp3_TEp3_VALp2 = [exp3_p1,exp3_p4,exp3_p5]
train_set_exp3_TEp3_VALp1 = [exp3_p2,exp3_p4,exp3_p5]

valid_set_exp3_TEp3_VALp5 = [exp3_p5]
valid_set_exp3_TEp3_VALp4 = [exp3_p4]
valid_set_exp3_TEp3_VALp2 = [exp3_p2]
valid_set_exp3_TEp3_VALp1 = [exp3_p1]

log_exp3_X_train_100_list = []
sqrt_exp3_X_train_100_list = []
log_exp3_X_test_100_list = []
sqrt_exp3_X_test_100_list = []

exp3_y_train_100_list = []
exp3_y_test_100_list = []

# generate training set for using validation set p4
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp3_VALp5,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp3_VALp5,sqrt_top_exp3_100['Features'].to_list())))
# p3
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp3_VALp4,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp3_VALp4,sqrt_top_exp3_100['Features'].to_list())))
# p2
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp3_VALp2,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp3_VALp2,sqrt_top_exp3_100['Features'].to_list())))
# p1
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp3_VALp1,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp3_VALp1,sqrt_top_exp3_100['Features'].to_list())))

# validation set p4
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp3_VALp5,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp3_VALp5,sqrt_top_exp3_100['Features'].to_list())))
# validation set p3
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp3_VALp4,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp3_VALp4,sqrt_top_exp3_100['Features'].to_list())))
# validation set p2
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp3_VALp2,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp3_VALp2,sqrt_top_exp3_100['Features'].to_list())))
# validation set p1
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp3_VALp1,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp3_VALp1,sqrt_top_exp3_100['Features'].to_list())))


exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp3_VALp5))
exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp3_VALp4))
exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp3_VALp2))
exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp3_VALp1))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp3_VALp5))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp3_VALp4))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp3_VALp2))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp3_VALp1))

In [207]:
train_exp3_TRp1245 = [exp3_p1,exp3_p2,exp3_p4,exp3_p5]
test_exp3_TEp3 = [exp3_p3]

log_exp3_X_train_100_TRp1245 = lib.rename_columns(lib.extract_data(train_exp3_TRp1245,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_train_100_TRp1245 = lib.rename_columns(lib.extract_data(train_exp3_TRp1245,sqrt_top_exp3_100['Features'].to_list()))
log_exp3_X_test_100_TEp3 = lib.rename_columns(lib.extract_data(test_exp3_TEp3,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_test_100_TEp3 = lib.rename_columns(lib.extract_data(test_exp3_TEp3,sqrt_top_exp3_100['Features'].to_list()))

exp3_y_train_100_TRp1245 = lib.extract_label(train_exp3_TRp1245)
exp3_y_test_100_TEp3 = lib.extract_label(test_exp3_TEp3)

In [208]:
output_file = open('./XGBobjective_weight_output_TEp3.txt', 'w')
for i in range(4):
    X_train, y_train, X_test, y_test = log_exp3_X_train_100_list[i], exp3_y_train_100_list[i], log_exp3_X_test_100_list[i], exp3_y_test_100_list[i]
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)

    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp3_VALp'+str(i+1)+'_averageTSSHSS'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp3_VALp'+str(i+1)+'_0.5TPR_1.5TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS2,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp3_VALp'+str(i+1)+'_0.75TPR_1.25TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS3,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp3_VALp'+str(i+1)+'_0.2TPR_1.8TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS4,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp3_VALp'+str(i+1)+'_1.5TPR_0.5TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)
    
output_file.close()

100%|████████| 50/50 [11:23<00:00, 13.66s/trial, best loss: -0.7403200013148215]


## Testing on p2

In [209]:
train_set_exp3_TEp2_VALp5 = [exp3_p1,exp3_p3,exp3_p4]
train_set_exp3_TEp2_VALp4 = [exp3_p1,exp3_p3,exp3_p5]
train_set_exp3_TEp2_VALp3 = [exp3_p1,exp3_p4,exp3_p5]
train_set_exp3_TEp2_VALp1 = [exp3_p3,exp3_p4,exp3_p5]

valid_set_exp3_TEp2_VALp5 = [exp3_p5]
valid_set_exp3_TEp2_VALp4 = [exp3_p4]
valid_set_exp3_TEp2_VALp3 = [exp3_p3]
valid_set_exp3_TEp2_VALp1 = [exp3_p1]

log_exp3_X_train_100_list = []
sqrt_exp3_X_train_100_list = []
log_exp3_X_test_100_list = []
sqrt_exp3_X_test_100_list = []

exp3_y_train_100_list = []
exp3_y_test_100_list = []

# generate training set for using validation set p4
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp2_VALp5,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp2_VALp5,sqrt_top_exp3_100['Features'].to_list())))
# p3
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp2_VALp4,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp2_VALp4,sqrt_top_exp3_100['Features'].to_list())))
# p2
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp2_VALp3,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp2_VALp3,sqrt_top_exp3_100['Features'].to_list())))
# p1
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp2_VALp1,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp2_VALp1,sqrt_top_exp3_100['Features'].to_list())))

# validation set p4
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp2_VALp5,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp2_VALp5,sqrt_top_exp3_100['Features'].to_list())))
# validation set p3
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp2_VALp4,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp2_VALp4,sqrt_top_exp3_100['Features'].to_list())))
# validation set p2
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp2_VALp3,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp2_VALp3,sqrt_top_exp3_100['Features'].to_list())))
# validation set p1
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp2_VALp1,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp2_VALp1,sqrt_top_exp3_100['Features'].to_list())))


exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp2_VALp5))
exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp2_VALp4))
exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp2_VALp3))
exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp2_VALp1))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp2_VALp5))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp2_VALp4))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp2_VALp3))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp2_VALp1))

In [210]:
train_exp3_TRp1345 = [exp3_p1,exp3_p3,exp3_p4,exp3_p5]
test_exp3_TEp2 = [exp3_p2]

log_exp3_X_train_100_TRp1345 = lib.rename_columns(lib.extract_data(train_exp3_TRp1345,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_train_100_TRp1345 = lib.rename_columns(lib.extract_data(train_exp3_TRp1345,sqrt_top_exp3_100['Features'].to_list()))
log_exp3_X_test_100_TEp2 = lib.rename_columns(lib.extract_data(test_exp3_TEp2,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_test_100_TEp2 = lib.rename_columns(lib.extract_data(test_exp3_TEp2,sqrt_top_exp3_100['Features'].to_list()))

exp3_y_train_100_TRp1345 = lib.extract_label(train_exp3_TRp1345)
exp3_y_test_100_TEp2 = lib.extract_label(test_exp3_TEp2)

In [211]:
output_file = open('./XGBobjective_weight_output_TEp2.txt', 'w')
for i in range(4):
    X_train, y_train, X_test, y_test = log_exp3_X_train_100_list[i], exp3_y_train_100_list[i], log_exp3_X_test_100_list[i], exp3_y_test_100_list[i]
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)

    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp2_VALp'+str(i+1)+'_averageTSSHSS'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp2_VALp'+str(i+1)+'_0.5TPR_1.5TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS2,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp2_VALp'+str(i+1)+'_0.75TPR_1.25TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS3,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp2_VALp'+str(i+1)+'_0.2TPR_1.8TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS4,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp2_VALp'+str(i+1)+'_1.5TPR_0.5TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)
    
output_file.close()

100%|████████| 50/50 [13:41<00:00, 16.43s/trial, best loss: -0.7693491269287038]


## Testing on p1

In [212]:
train_set_exp3_TEp1_VALp5 = [exp3_p2,exp3_p3,exp3_p4]
train_set_exp3_TEp1_VALp4 = [exp3_p2,exp3_p3,exp3_p5]
train_set_exp3_TEp1_VALp3 = [exp3_p2,exp3_p4,exp3_p5]
train_set_exp3_TEp1_VALp2 = [exp3_p3,exp3_p4,exp3_p5]

valid_set_exp3_TEp1_VALp5 = [exp3_p5]
valid_set_exp3_TEp1_VALp4 = [exp3_p4]
valid_set_exp3_TEp1_VALp3 = [exp3_p3]
valid_set_exp3_TEp1_VALp2 = [exp3_p2]

log_exp3_X_train_100_list = []
sqrt_exp3_X_train_100_list = []
log_exp3_X_test_100_list = []
sqrt_exp3_X_test_100_list = []

exp3_y_train_100_list = []
exp3_y_test_100_list = []

# generate training set for using validation set p4
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp1_VALp5,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp1_VALp5,sqrt_top_exp3_100['Features'].to_list())))
# p3
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp1_VALp4,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp1_VALp4,sqrt_top_exp3_100['Features'].to_list())))
# p2
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp1_VALp3,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp1_VALp3,sqrt_top_exp3_100['Features'].to_list())))
# p1
log_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp1_VALp2,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_train_100_list.append(lib.rename_columns(lib.extract_data(train_set_exp3_TEp1_VALp2,sqrt_top_exp3_100['Features'].to_list())))

# validation set p4
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp1_VALp5,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp1_VALp5,sqrt_top_exp3_100['Features'].to_list())))
# validation set p3
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp1_VALp4,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp1_VALp4,sqrt_top_exp3_100['Features'].to_list())))
# validation set p2
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp1_VALp3,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp1_VALp3,sqrt_top_exp3_100['Features'].to_list())))
# validation set p1
log_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp1_VALp2,log2_top_exp3_100['Features'].to_list())))
sqrt_exp3_X_test_100_list.append(lib.rename_columns(lib.extract_data(valid_set_exp3_TEp1_VALp2,sqrt_top_exp3_100['Features'].to_list())))


exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp1_VALp5))
exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp1_VALp4))
exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp1_VALp3))
exp3_y_train_100_list.append(lib.extract_label(train_set_exp3_TEp1_VALp2))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp1_VALp5))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp1_VALp4))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp1_VALp3))
exp3_y_test_100_list.append(lib.extract_label(valid_set_exp3_TEp1_VALp2))

In [214]:
train_exp3_TRp2345 = [exp3_p2,exp3_p3,exp3_p4,exp3_p5]
test_exp3_TEp1 = [exp3_p1]

log_exp3_X_train_100_TRp2345 = lib.rename_columns(lib.extract_data(train_exp3_TRp2345,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_train_100_TRp2345 = lib.rename_columns(lib.extract_data(train_exp3_TRp2345,sqrt_top_exp3_100['Features'].to_list()))
log_exp3_X_test_100_TEp1 = lib.rename_columns(lib.extract_data(test_exp3_TEp1,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_test_100_TEp1 = lib.rename_columns(lib.extract_data(test_exp3_TEp1,sqrt_top_exp3_100['Features'].to_list()))

exp3_y_train_100_TRp2345 = lib.extract_label(train_exp3_TRp2345)
exp3_y_test_100_TEp1 = lib.extract_label(test_exp3_TEp1)

In [215]:
output_file = open('./XGBobjective_weight_output_TEp1.txt', 'w')
for i in range(4):
    X_train, y_train, X_test, y_test = log_exp3_X_train_100_list[i], exp3_y_train_100_list[i], log_exp3_X_test_100_list[i], exp3_y_test_100_list[i]
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)

    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp1_VALp'+str(i+1)+'_averageTSSHSS'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp1_VALp'+str(i+1)+'_0.5TPR_1.5TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS2,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp1_VALp'+str(i+1)+'_0.75TPR_1.25TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS3,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp1_VALp'+str(i+1)+'_0.2TPR_1.8TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)


    trials = Trials()
    best_hyperparams_weighted = fmin(fn = objective_weighted_TSS4,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 50,
                            trials = trials)
    
    best_hyperparams_weighted['TSS'], best_hyperparams_weighted['HSS'], best_hyperparams_weighted['Confusion_Matrix(tn, fp, fn, tp)'], best_hyperparams_weighted['score'] = getBestModelfromTrials(trials)
    best_hyperparams_weighted['exp'] = 'log2_exp3_boostrap100_TEp1_VALp'+str(i+1)+'_1.5TPR_0.5TNR'
    hp_df = hp_df.append(best_hyperparams_weighted, ignore_index=True)
    
output_file.close()

100%|████████| 50/50 [00:52<00:00,  1.04s/trial, best loss: -0.8015399054964811]


In [218]:
hp_df['composite_index'] = 0
for i in range(len(hp_df)):
    tss = hp_df['TSS'][i]
    hss = hp_df['HSS'][i]
    if tss != 0 and hss != 0:
        hp_df.loc[i,'composite_index'] = math.sqrt(tss*hss)

In [ ]:
hp_df.to_csv('./XGBoost_internal_CV.csv')

In [232]:
hp_df = pd.read_csv('./XGBoost_internal_CV.csv',index_col=0)

# Final testing deployment

In [220]:
train_set_exp3_TEp5 = [exp3_p1,exp3_p2,exp3_p3,exp3_p4]
train_set_exp3_TEp4 = [exp3_p1,exp3_p2,exp3_p3,exp3_p5]
train_set_exp3_TEp3 = [exp3_p1,exp3_p2,exp3_p4,exp3_p5]
train_set_exp3_TEp2 = [exp3_p1,exp3_p3,exp3_p4,exp3_p5]
train_set_exp3_TEp1 = [exp3_p2,exp3_p3,exp3_p4,exp3_p5]

train_set_exp3_list = [train_set_exp3_TEp1, train_set_exp3_TEp2, train_set_exp3_TEp3, train_set_exp3_TEp4, train_set_exp3_TEp5]

In [221]:
test_set_exp3_TEp5 = [exp3_p5]
test_set_exp3_TEp4 = [exp3_p4]
test_set_exp3_TEp3 = [exp3_p3]
test_set_exp3_TEp2 = [exp3_p2]
test_set_exp3_TEp1 = [exp3_p1]

test_set_exp3_list = [test_set_exp3_TEp1, test_set_exp3_TEp2, test_set_exp3_TEp3, test_set_exp3_TEp4, test_set_exp3_TEp5]

In [275]:
hp_df['Testing_TSS'] = ''
hp_df['Testing_HSS'] = ''
hp_df['Testing_CMatrix'] = ''
for i in range(len(hp_df)):
    bst = XGBClassifier(n_estimators = int(hp_df['n_estimators'][i]), max_depth = int(hp_df['max_depth'][i]), gamma = hp_df['gamma'][i],
                    reg_alpha = int(hp_df['reg_alpha'][i]),min_child_weight=int(hp_df['min_child_weight'][i]), scale_pos_weight=hp_df['scale_pos_weight'][i],
                    colsample_bytree=int(hp_df['colsample_bytree'][i]))
    t0=time.time()
    if 'TEp5' in hp_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp5, exp3_y_train_100_TEp5)
        y_pred = bst.predict(log_exp3_X_test_100_TEp5)
        scores = confusion_matrix(exp3_y_test_100_TEp5, y_pred, labels=[0, 1]).ravel()
        hp_df['Testing_TSS'][i] = measurements.TSS(scores)
        hp_df['Testing_HSS'][i] = measurements.HSS2(scores)
        hp_df['Testing_CMatrix'][i] = [scores]
        
    if 'TEp4' in hp_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp4, exp3_y_train_100_TEp4)
        y_pred = bst.predict(log_exp3_X_test_100_TEp4)
        scores = confusion_matrix(exp3_y_test_100_TEp4, y_pred, labels=[0, 1]).ravel()
        hp_df['Testing_TSS'][i] = measurements.TSS(scores)
        hp_df['Testing_HSS'][i] = measurements.HSS2(scores)
        hp_df['Testing_CMatrix'][i] = [scores]
        
    if 'TEp3' in hp_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp3, exp3_y_train_100_TEp3)
        y_pred = bst.predict(log_exp3_X_test_100_TEp3)
        scores = confusion_matrix(exp3_y_test_100_TEp3, y_pred, labels=[0, 1]).ravel()
        hp_df['Testing_TSS'][i] = measurements.TSS(scores)
        hp_df['Testing_HSS'][i] = measurements.HSS2(scores)
        hp_df['Testing_CMatrix'][i] = [scores]
        
    if 'TEp2' in hp_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp2, exp3_y_train_100_TEp2)
        y_pred = bst.predict(log_exp3_X_test_100_TEp2)
        scores = confusion_matrix(exp3_y_test_100_TEp2, y_pred, labels=[0, 1]).ravel()
        hp_df['Testing_TSS'][i] = measurements.TSS(scores)
        hp_df['Testing_HSS'][i] = measurements.HSS2(scores)
        hp_df['Testing_CMatrix'][i] = [scores]
        
    if 'TEp1' in hp_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp1, exp3_y_train_100_TEp1)
        y_pred = bst.predict(log_exp3_X_test_100_TEp1)
        scores = confusion_matrix(exp3_y_test_100_TEp1, y_pred, labels=[0, 1]).ravel()
        hp_df['Testing_TSS'][i] = measurements.TSS(scores)
        hp_df['Testing_HSS'][i] = measurements.HSS2(scores)
        hp_df['Testing_CMatrix'][i] = [scores]

In [238]:
#p5
log_exp3_X_train_100_TEp5 = lib.rename_columns(lib.extract_data(train_set_exp3_TEp5,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_train_100_TEp5 = lib.rename_columns(lib.extract_data(train_set_exp3_TEp5,sqrt_top_exp3_100['Features'].to_list()))
log_exp3_X_test_100_TEp5 = lib.rename_columns(lib.extract_data(test_set_exp3_TEp5,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_test_100_TEp5 = lib.rename_columns(lib.extract_data(test_set_exp3_TEp5,sqrt_top_exp3_100['Features'].to_list()))

exp3_y_train_100_TEp5 = lib.extract_label(train_set_exp3_TEp5)
exp3_y_test_100_TEp5 = lib.extract_label(test_set_exp3_TEp5)

#p4
log_exp3_X_train_100_TEp4 = lib.rename_columns(lib.extract_data(train_set_exp3_TEp4,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_train_100_TEp4 = lib.rename_columns(lib.extract_data(train_set_exp3_TEp4,sqrt_top_exp3_100['Features'].to_list()))
log_exp3_X_test_100_TEp4 = lib.rename_columns(lib.extract_data(test_set_exp3_TEp4,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_test_100_TEp4 = lib.rename_columns(lib.extract_data(test_set_exp3_TEp4,sqrt_top_exp3_100['Features'].to_list()))

exp3_y_train_100_TEp4 = lib.extract_label(train_set_exp3_TEp4)
exp3_y_test_100_TEp4 = lib.extract_label(test_set_exp3_TEp4)

#p3
log_exp3_X_train_100_TEp3 = lib.rename_columns(lib.extract_data(train_set_exp3_TEp3,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_train_100_TEp3 = lib.rename_columns(lib.extract_data(train_set_exp3_TEp3,sqrt_top_exp3_100['Features'].to_list()))
log_exp3_X_test_100_TEp3 = lib.rename_columns(lib.extract_data(test_set_exp3_TEp3,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_test_100_TEp3 = lib.rename_columns(lib.extract_data(test_set_exp3_TEp3,sqrt_top_exp3_100['Features'].to_list()))

exp3_y_train_100_TEp3 = lib.extract_label(train_set_exp3_TEp3)
exp3_y_test_100_TEp3 = lib.extract_label(test_set_exp3_TEp3)

#p2
log_exp3_X_train_100_TEp2 = lib.rename_columns(lib.extract_data(train_set_exp3_TEp2,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_train_100_TEp2 = lib.rename_columns(lib.extract_data(train_set_exp3_TEp2,sqrt_top_exp3_100['Features'].to_list()))
log_exp3_X_test_100_TEp2 = lib.rename_columns(lib.extract_data(test_set_exp3_TEp2,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_test_100_TEp2 = lib.rename_columns(lib.extract_data(test_set_exp3_TEp2,sqrt_top_exp3_100['Features'].to_list()))

exp3_y_train_100_TEp2 = lib.extract_label(train_set_exp3_TEp2)
exp3_y_test_100_TEp2 = lib.extract_label(test_set_exp3_TEp2)

#p1
log_exp3_X_train_100_TEp1 = lib.rename_columns(lib.extract_data(train_set_exp3_TEp1,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_train_100_TEp1 = lib.rename_columns(lib.extract_data(train_set_exp3_TEp1,sqrt_top_exp3_100['Features'].to_list()))
log_exp3_X_test_100_TEp1 = lib.rename_columns(lib.extract_data(test_set_exp3_TEp1,log2_top_exp3_100['Features'].to_list()))
sqrt_exp3_X_test_100_TEp1 = lib.rename_columns(lib.extract_data(test_set_exp3_TEp1,sqrt_top_exp3_100['Features'].to_list()))

exp3_y_train_100_TEp1 = lib.extract_label(train_set_exp3_TEp1)
exp3_y_test_100_TEp1 = lib.extract_label(test_set_exp3_TEp1)

In [260]:
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [261]:
max_composite = []
max_TSS = []
max_HSS = []
for i in range(0,len(hp_df),5):
    sub_df = hp_df[i:i+5]
    max_composite.append(sub_df['composite_index'].idxmax())
    max_TSS.append(sub_df['TSS'].idxmax())
    max_HSS.append(sub_df['HSS'].idxmax())
    
max_composite_df = hp_df.iloc[max_composite]
max_composite_df = max_composite_df.reset_index(drop=True)
max_TSS_df = hp_df.iloc[max_TSS]
max_TSS_df = max_TSS_df.reset_index(drop=True)
max_HSS_df = hp_df.iloc[max_HSS]
max_HSS_df = max_HSS_df.reset_index(drop=True)

In [262]:
max_composite_df['Testing_TSS'] = ''
max_composite_df['Testing_HSS'] = ''
max_composite_df['Testing_CMatrix'] = ''

for i in range(len(max_composite_df)):
    bst = XGBClassifier(n_estimators = int(max_composite_df['n_estimators'][i]), max_depth = int(max_composite_df['max_depth'][i]), gamma = max_composite_df['gamma'][i],
                    reg_alpha = int(max_composite_df['reg_alpha'][i]),min_child_weight=int(max_composite_df['min_child_weight'][i]), scale_pos_weight=max_composite_df['scale_pos_weight'][i],
                    colsample_bytree=int(max_composite_df['colsample_bytree'][i]))
    t0=time.time()
    if 'TEp5' in max_composite_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp5, exp3_y_train_100_TEp5)
        y_pred = bst.predict(log_exp3_X_test_100_TEp5)
        scores = confusion_matrix(exp3_y_test_100_TEp5, y_pred, labels=[0, 1]).ravel()
        max_composite_df['Testing_TSS'][i] = measurements.TSS(scores)
        max_composite_df['Testing_HSS'][i] = measurements.HSS2(scores)
        max_composite_df['Testing_CMatrix'][i] = [scores]
        
    if 'TEp4' in max_composite_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp4, exp3_y_train_100_TEp4)
        y_pred = bst.predict(log_exp3_X_test_100_TEp4)
        scores = confusion_matrix(exp3_y_test_100_TEp4, y_pred, labels=[0, 1]).ravel()
        max_composite_df['Testing_TSS'][i] = measurements.TSS(scores)
        max_composite_df['Testing_HSS'][i] = measurements.HSS2(scores)
        max_composite_df['Testing_CMatrix'][i] = [scores]
        
    if 'TEp3' in max_composite_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp3, exp3_y_train_100_TEp3)
        y_pred = bst.predict(log_exp3_X_test_100_TEp3)
        scores = confusion_matrix(exp3_y_test_100_TEp3, y_pred, labels=[0, 1]).ravel()
        max_composite_df['Testing_TSS'][i] = measurements.TSS(scores)
        max_composite_df['Testing_HSS'][i] = measurements.HSS2(scores)
        max_composite_df['Testing_CMatrix'][i] = [scores]
        
    if 'TEp2' in max_composite_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp2, exp3_y_train_100_TEp2)
        y_pred = bst.predict(log_exp3_X_test_100_TEp2)
        scores = confusion_matrix(exp3_y_test_100_TEp2, y_pred, labels=[0, 1]).ravel()
        max_composite_df['Testing_TSS'][i] = measurements.TSS(scores)
        max_composite_df['Testing_HSS'][i] = measurements.HSS2(scores)
        max_composite_df['Testing_CMatrix'][i] = [scores]
        
    if 'TEp1' in max_composite_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp1, exp3_y_train_100_TEp1)
        y_pred = bst.predict(log_exp3_X_test_100_TEp1)
        scores = confusion_matrix(exp3_y_test_100_TEp1, y_pred, labels=[0, 1]).ravel()
        max_composite_df['Testing_TSS'][i] = measurements.TSS(scores)
        max_composite_df['Testing_HSS'][i] = measurements.HSS2(scores)
        max_composite_df['Testing_CMatrix'][i] = [scores]

In [263]:
# Maximum on composite index
max_composite_df

,colsample_bytree,gamma,max_depth,min_child_weight,n_estimators,reg_alpha,reg_lambda,scale_pos_weight,TSS,HSS,"Confusion_Matrix(tn, fp, fn, tp)",score,exp,composite_index,Testing_TSS,Testing_HSS,Testing_CMatrix
0,0.506493,8.786948,6.0,3.0,36.0,106.0,0.009724,3,0.595809,0.459004,"[array([48899, 1155, 444, 721])]",0.882242,log2_exp3_boostrap100_TEp5_VALp1_0.2TPR_1.8TNR,0.522952,0.413311,0.426488,"[[73790, 512, 574, 416]]"
1,0.612135,1.021725,8.0,1.0,83.0,179.0,0.175087,7,0.693291,0.358954,"[array([37980, 3078, 330, 1094])]",0.771679,log3_exp3_boostrap100_TEp5_VALp2_0.5TPR_1.5TNR,0.498858,0.686089,0.410772,"[[72668, 1634, 289, 701]]"
2,0.663236,8.621182,7.0,2.0,56.0,180.0,0.135506,8,0.705567,0.307506,"[array([82249, 4034, 347, 1054])]",0.506536,log2_exp3_boostrap100_TEp5_VALp3_averageTSSHSS,0.465796,0.734039,0.393463,"[[72328, 1974, 237, 753]]"
3,0.827642,6.862093,10.0,6.0,86.0,144.0,0.982012,4,0.600936,0.380484,"[array([70291, 1947, 467, 788])]",0.877063,log2_exp3_boostrap100_TEp5_VALp4_0.2TPR_1.8TNR,0.478170,0.482378,0.422943,"[[73518, 784, 502, 488]]"
4,0.675575,1.008028,3.0,9.0,67.0,126.0,0.253709,8,0.776331,0.379998,"[array([72018, 2284, 191, 799])]",0.906083,log2_exp3_boostrap100_TEp4_VALp1_0.2TPR_1.8TNR,0.543143,0.691671,0.393839,"[[47940, 2114, 310, 855]]"
5,0.804962,2.695549,12.0,7.0,11.0,120.0,0.787761,9,0.760155,0.319914,"[array([36804, 4254, 194, 1230])]",0.776468,log2_exp3_boostrap100_TEp4_VALp2_0.5TPR_1.5TNR,0.493137,0.704286,0.390217,"[[47841, 2213, 293, 872]]"
6,0.860424,2.031684,18.0,8.0,84.0,159.0,0.601518,3,0.549194,0.362240,"[array([84215, 2068, 598, 803])]",0.871491,log2_exp3_boostrap100_TEp4_VALp3_0.2TPR_1.8TNR,0.446027,0.508204,0.490918,"[[49412, 642, 558, 607]]"
7,0.675675,7.723013,14.0,2.0,11.0,150.0,0.661513,6,0.658432,0.363699,"[array([69782, 2456, 386, 869])]",0.877288,log2_exp3_boostrap100_TEp4_VALp5_0.2TPR_1.8TNR,0.489358,0.631516,0.434961,"[[48581, 1473, 395, 770]]"
8,0.652145,2.735663,11.0,3.0,27.0,125.0,0.754978,7,0.772141,0.399481,"[array([72232, 2070, 198, 792])]",0.909853,log2_exp3_boostrap100_TEp3_VALp1_0.2TPR_1.8TNR,0.555388,0.644008,0.370643,"[[38465, 2593, 417, 1007]]"
9,0.865246,4.912579,18.0,8.0,80.0,160.0,0.135026,2,0.587168,0.480759,"[array([49068, 986, 458, 707])]",0.885916,log2_exp3_boostrap100_TEp3_VALp2_0.2TPR_1.8TNR,0.531306,0.39176,0.416613,"[[40391, 667, 843, 581]]"


In [264]:
max_TSS_df['Testing_TSS'] = ''
max_TSS_df['Testing_HSS'] = ''
max_TSS_df['Testing_CMatrix'] = ''

for i in range(len(max_TSS_df)):
    bst = XGBClassifier(n_estimators = int(max_TSS_df['n_estimators'][i]), max_depth = int(max_TSS_df['max_depth'][i]), gamma = max_TSS_df['gamma'][i],
                    reg_alpha = int(max_TSS_df['reg_alpha'][i]),min_child_weight=int(max_TSS_df['min_child_weight'][i]), scale_pos_weight=max_TSS_df['scale_pos_weight'][i],
                    colsample_bytree=int(max_TSS_df['colsample_bytree'][i]))
    t0=time.time()
    if 'TEp5' in max_TSS_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp5, exp3_y_train_100_TEp5)
        y_pred = bst.predict(log_exp3_X_test_100_TEp5)
        scores = confusion_matrix(exp3_y_test_100_TEp5, y_pred, labels=[0, 1]).ravel()
        max_TSS_df['Testing_TSS'][i] = measurements.TSS(scores)
        max_TSS_df['Testing_HSS'][i] = measurements.HSS2(scores)
        max_TSS_df['Testing_CMatrix'][i] = [scores]
        
    if 'TEp4' in max_TSS_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp4, exp3_y_train_100_TEp4)
        y_pred = bst.predict(log_exp3_X_test_100_TEp4)
        scores = confusion_matrix(exp3_y_test_100_TEp4, y_pred, labels=[0, 1]).ravel()
        max_TSS_df['Testing_TSS'][i] = measurements.TSS(scores)
        max_TSS_df['Testing_HSS'][i] = measurements.HSS2(scores)
        max_TSS_df['Testing_CMatrix'][i] = [scores]
        
    if 'TEp3' in max_TSS_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp3, exp3_y_train_100_TEp3)
        y_pred = bst.predict(log_exp3_X_test_100_TEp3)
        scores = confusion_matrix(exp3_y_test_100_TEp3, y_pred, labels=[0, 1]).ravel()
        max_TSS_df['Testing_TSS'][i] = measurements.TSS(scores)
        max_TSS_df['Testing_HSS'][i] = measurements.HSS2(scores)
        max_TSS_df['Testing_CMatrix'][i] = [scores]
        
    if 'TEp2' in max_TSS_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp2, exp3_y_train_100_TEp2)
        y_pred = bst.predict(log_exp3_X_test_100_TEp2)
        scores = confusion_matrix(exp3_y_test_100_TEp2, y_pred, labels=[0, 1]).ravel()
        max_TSS_df['Testing_TSS'][i] = measurements.TSS(scores)
        max_TSS_df['Testing_HSS'][i] = measurements.HSS2(scores)
        max_TSS_df['Testing_CMatrix'][i] = [scores]
        
    if 'TEp1' in max_TSS_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp1, exp3_y_train_100_TEp1)
        y_pred = bst.predict(log_exp3_X_test_100_TEp1)
        scores = confusion_matrix(exp3_y_test_100_TEp1, y_pred, labels=[0, 1]).ravel()
        max_TSS_df['Testing_TSS'][i] = measurements.TSS(scores)
        max_TSS_df['Testing_HSS'][i] = measurements.HSS2(scores)
        max_TSS_df['Testing_CMatrix'][i] = [scores]

In [265]:
# Maximum on TSS
max_TSS_df

,colsample_bytree,gamma,max_depth,min_child_weight,n_estimators,reg_alpha,reg_lambda,scale_pos_weight,TSS,HSS,"Confusion_Matrix(tn, fp, fn, tp)",score,exp,composite_index,Testing_TSS,Testing_HSS,Testing_CMatrix
0,0.908068,8.845125,3.0,4.0,12.0,72.0,0.007414,9,0.838896,0.304110,"[array([45771, 4283, 88, 1077])]",0.836388,log2_exp3_boostrap100_TEp5_VALp1_0.75TPR_1.25TNR,0.505091,0.757219,0.380005,"[[72099, 2203, 211, 779]]"
1,0.782940,6.577603,9.0,6.0,12.0,179.0,0.040286,10,0.785236,0.290338,"[array([35902, 5156, 127, 1297])]",0.537787,log2_exp3_boostrap100_TEp5_VALp2_averageTSSHSS,0.477477,0.793774,0.37491,"[[71888, 2414, 172, 818]]"
2,0.525040,6.837140,3.0,9.0,37.0,74.0,0.194993,10,0.807628,0.205657,"[array([77814, 8469, 132, 1269])]",0.806644,log2_exp3_boostrap100_TEp5_VALp3_0.75TPR_1.25TNR,0.407547,0.795714,0.36854,"[[71807, 2495, 169, 821]]"
3,0.759101,3.605940,10.0,8.0,11.0,78.0,0.843262,10,0.779594,0.232112,"[array([66044, 6194, 169, 1086])]",0.791824,log2_exp3_boostrap100_TEp5_VALp4_0.75TPR_1.25TNR,0.425386,0.782325,0.374975,"[[71938, 2364, 184, 806]]"
4,0.515467,5.952084,13.0,4.0,35.0,117.0,0.228748,10,0.869902,0.268492,"[array([69739, 4563, 68, 922])]",0.866264,log2_exp3_boostrap100_TEp4_VALp1_1.5TPR_0.5TNR,0.483282,0.719932,0.375184,"[[47593, 2461, 269, 896]]"
5,0.592627,4.198975,4.0,10.0,41.0,121.0,0.426118,10,0.800507,0.299361,"[array([36010, 5048, 109, 1315])]",0.549934,log2_exp3_boostrap100_TEp4_VALp2_averageTSSHSS,0.489531,0.718495,0.375749,"[[47607, 2447, 271, 894]]"
6,0.982855,8.239800,4.0,0.0,12.0,74.0,0.705050,10,0.807710,0.211735,"[array([78129, 8154, 137, 1264])]",0.509723,log2_exp3_boostrap100_TEp4_VALp3_averageTSSHSS,0.413547,0.729813,0.377611,"[[47572, 2482, 257, 908]]"
7,0.990514,4.231479,9.0,6.0,12.0,132.0,0.221188,10,0.790773,0.230392,"[array([65873, 6365, 152, 1103])]",0.774271,log2_exp3_boostrap100_TEp4_VALp5_1.5TPR_0.5TNR,0.426834,0.733286,0.379244,"[[47574, 2480, 253, 912]]"
8,0.835974,6.897669,12.0,3.0,19.0,168.0,0.452969,10,0.863260,0.274506,"[array([69921, 4381, 77, 913])]",0.867964,log2_exp3_boostrap100_TEp3_VALp1_0.75TPR_1.25TNR,0.486796,0.701406,0.352394,"[[37823, 3235, 313, 1111]]"
9,0.889376,4.385649,13.0,8.0,10.0,121.0,0.982514,10,0.842344,0.291580,"[array([45471, 4583, 77, 1088])]",0.855078,log2_exp3_boostrap100_TEp3_VALp2_1.5TPR_0.5TNR,0.495592,0.687754,0.349464,"[[37868, 3190, 334, 1090]]"


In [266]:
max_HSS_df['Testing_TSS'] = ''
max_HSS_df['Testing_HSS'] = ''
max_HSS_df['Testing_CMatrix'] = ''

for i in range(len(max_HSS_df)):
    bst = XGBClassifier(n_estimators = int(max_HSS_df['n_estimators'][i]), max_depth = int(max_HSS_df['max_depth'][i]), gamma = max_HSS_df['gamma'][i],
                    reg_alpha = int(max_HSS_df['reg_alpha'][i]),min_child_weight=int(max_HSS_df['min_child_weight'][i]), scale_pos_weight=max_HSS_df['scale_pos_weight'][i],
                    colsample_bytree=int(max_HSS_df['colsample_bytree'][i]))
    t0=time.time()
    if 'TEp5' in max_HSS_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp5, exp3_y_train_100_TEp5)
        y_pred = bst.predict(log_exp3_X_test_100_TEp5)
        scores = confusion_matrix(exp3_y_test_100_TEp5, y_pred, labels=[0, 1]).ravel()
        max_HSS_df['Testing_TSS'][i] = measurements.TSS(scores)
        max_HSS_df['Testing_HSS'][i] = measurements.HSS2(scores)
        max_HSS_df['Testing_CMatrix'][i] = [scores]
        
    if 'TEp4' in max_HSS_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp4, exp3_y_train_100_TEp4)
        y_pred = bst.predict(log_exp3_X_test_100_TEp4)
        scores = confusion_matrix(exp3_y_test_100_TEp4, y_pred, labels=[0, 1]).ravel()
        max_HSS_df['Testing_TSS'][i] = measurements.TSS(scores)
        max_HSS_df['Testing_HSS'][i] = measurements.HSS2(scores)
        max_HSS_df['Testing_CMatrix'][i] = [scores]
        
    if 'TEp3' in max_HSS_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp3, exp3_y_train_100_TEp3)
        y_pred = bst.predict(log_exp3_X_test_100_TEp3)
        scores = confusion_matrix(exp3_y_test_100_TEp3, y_pred, labels=[0, 1]).ravel()
        max_HSS_df['Testing_TSS'][i] = measurements.TSS(scores)
        max_HSS_df['Testing_HSS'][i] = measurements.HSS2(scores)
        max_HSS_df['Testing_CMatrix'][i] = [scores]
        
    if 'TEp2' in max_HSS_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp2, exp3_y_train_100_TEp2)
        y_pred = bst.predict(log_exp3_X_test_100_TEp2)
        scores = confusion_matrix(exp3_y_test_100_TEp2, y_pred, labels=[0, 1]).ravel()
        max_HSS_df['Testing_TSS'][i] = measurements.TSS(scores)
        max_HSS_df['Testing_HSS'][i] = measurements.HSS2(scores)
        max_HSS_df['Testing_CMatrix'][i] = [scores]
        
    if 'TEp1' in max_HSS_df['exp'][i]:
        bst.fit(log_exp3_X_train_100_TEp1, exp3_y_train_100_TEp1)
        y_pred = bst.predict(log_exp3_X_test_100_TEp1)
        scores = confusion_matrix(exp3_y_test_100_TEp1, y_pred, labels=[0, 1]).ravel()
        max_HSS_df['Testing_TSS'][i] = measurements.TSS(scores)
        max_HSS_df['Testing_HSS'][i] = measurements.HSS2(scores)
        max_HSS_df['Testing_CMatrix'][i] = [scores]

In [267]:
max_HSS_df

,colsample_bytree,gamma,max_depth,min_child_weight,n_estimators,reg_alpha,reg_lambda,scale_pos_weight,TSS,HSS,"Confusion_Matrix(tn, fp, fn, tp)",score,exp,composite_index,Testing_TSS,Testing_HSS,Testing_CMatrix
0,0.506493,8.786948,6.0,3.0,36.0,106.0,0.009724,3,0.595809,0.459004,"[array([48899, 1155, 444, 721])]",0.882242,log2_exp3_boostrap100_TEp5_VALp1_0.2TPR_1.8TNR,0.522952,0.413311,0.426488,"[[73790, 512, 574, 416]]"
1,0.993868,5.234716,4.0,8.0,41.0,59.0,0.315149,1,0.455186,0.421869,"[array([40083, 975, 742, 682])]",0.853042,log2_exp3_boostrap100_TEp5_VALp2_0.2TPR_1.8TNR,0.438211,0.14163,0.233587,"[[74243, 59, 849, 141]]"
2,0.996998,5.276356,12.0,8.0,94.0,154.0,0.324010,4,0.591672,0.356946,"[array([83877, 2406, 533, 868])]",0.873719,log2_exp3_boostrap100_TEp5_VALp3_0.2TPR_1.8TNR,0.459560,0.484331,0.42336,"[[73513, 789, 500, 490]]"
3,0.827642,6.862093,10.0,6.0,86.0,144.0,0.982012,4,0.600936,0.380484,"[array([70291, 1947, 467, 788])]",0.877063,log2_exp3_boostrap100_TEp5_VALp4_0.2TPR_1.8TNR,0.478170,0.482378,0.422943,"[[73518, 784, 502, 488]]"
4,0.675575,1.008028,3.0,9.0,67.0,126.0,0.253709,8,0.776331,0.379998,"[array([72018, 2284, 191, 799])]",0.906083,log2_exp3_boostrap100_TEp4_VALp1_0.2TPR_1.8TNR,0.543143,0.691671,0.393839,"[[47940, 2114, 310, 855]]"
5,0.997381,5.318395,12.0,0.0,11.0,57.0,0.549679,1,0.450562,0.420091,"[array([40095, 963, 749, 675])]",0.852585,log2_exp3_boostrap100_TEp4_VALp2_0.2TPR_1.8TNR,0.435060,0.243616,0.373366,"[[50003, 51, 880, 285]]"
6,0.860424,2.031684,18.0,8.0,84.0,159.0,0.601518,3,0.549194,0.362240,"[array([84215, 2068, 598, 803])]",0.871491,log2_exp3_boostrap100_TEp4_VALp3_0.2TPR_1.8TNR,0.446027,0.508204,0.490918,"[[49412, 642, 558, 607]]"
7,0.675675,7.723013,14.0,2.0,11.0,150.0,0.661513,6,0.658432,0.363699,"[array([69782, 2456, 386, 869])]",0.877288,log2_exp3_boostrap100_TEp4_VALp5_0.2TPR_1.8TNR,0.489358,0.631516,0.434961,"[[48581, 1473, 395, 770]]"
8,0.652145,2.735663,11.0,3.0,27.0,125.0,0.754978,7,0.772141,0.399481,"[array([72232, 2070, 198, 792])]",0.909853,log2_exp3_boostrap100_TEp3_VALp1_0.2TPR_1.8TNR,0.555388,0.644008,0.370643,"[[38465, 2593, 417, 1007]]"
9,0.865246,4.912579,18.0,8.0,80.0,160.0,0.135026,2,0.587168,0.480759,"[array([49068, 986, 458, 707])]",0.885916,log2_exp3_boostrap100_TEp3_VALp2_0.2TPR_1.8TNR,0.531306,0.39176,0.416613,"[[40391, 667, 843, 581]]"


In [269]:
max_composite_df.to_csv('./XGBoost_max_composite.csv')
max_TSS_df.to_csv('./XGBoost_max_TSS.csv')
max_HSS_df.to_csv('./XGBoost_max_HSS.csv')